<center>
<h1 style="font-size: 50px; font-family: serif;">Two Fold Dashboard</h1>
</center>


# Define the HTML content

<html>
<head>
    <title>Documentation</title>
    <style>
        body {font-family: Arial, sans-serif; margin: 20px;}
        h1 {color: #333;}
        h2 {color: #555;}
        ul {line-height: 1.8;}
    </style>
</head>
<body>


<p>This module is a Dash web application for visualizing battery cell data.</p>

<h2>Features:</h2>
<ul>
    <li>Displays a dashboard with State of Health (SoH) for different battery cells.</li>
    <li>Provides interactive visualizations including pie charts and line graphs.</li>
    <li>Supports dynamic updates based on user interactions with dropdowns and buttons.</li>
</ul>

<h2>Dependencies:</h2>
<ul>
    <li>Dash</li>
    <li>Plotly</li>
    <li>Pandas</li>
    <li>Requests</li>
    <li>Flask</li>
</ul>

<h2>Functions:</h2>
<ul>
    <li><strong>fetch_cell_data(cell_id)</strong>: Fetches data for the specified cell ID from the REST API.</li>
    <li><strong>toggle_dropdown_visibility(n_clicks)</strong>: Toggles the visibility of the dropdown menu.</li>
    <li><strong>display_page(dashboard_n_clicks, cell_id)</strong>: Displays the appropriate page content based on user interactions.</li>
    <li><strong>create_cell_visualizations(cell_id)</strong>: Generates visualizations for the selected cell ID.</li>
</ul>

<h2>Usage:</h2>
<p>Run this script to start the Dash server. Access the dashboard at <a href="http://localhost:8080">http://localhost:8080</a>.</p>

</body>
</html>



# import important libraries

In [1]:
import pandas as pd
import numpy as np

# Define function and load excel file

In [2]:
def load_excel_files(file_paths, cell_id):
    dfs = {}
    for file_path in file_paths:
        sheet_names = pd.ExcelFile(file_path).sheet_names
        for sheet_name in sheet_names:
            df = pd.read_excel(file_path, sheet_name=sheet_name)
            df['CellId'] = cell_id
            if sheet_name in dfs:
                dfs[sheet_name] = pd.concat([dfs[sheet_name], df], ignore_index=True)
            else:
                dfs[sheet_name] = df
    return dfs

# Define file path and then load the excel files

In [3]:
file_paths_5308 = ['5308.xls']
file_paths_5329 = ['5329.xls']

dfs_5308 = load_excel_files(file_paths_5308, '5308')
dfs_5329 = load_excel_files(file_paths_5329, '5329')


# Calculate the State of Health

In [4]:
#Define Discharge Capacities and Nominal Capacity
discharge_capacities = {'5308': 2992.02, '5329': 2822.56}
nominal_capacity = 3000

#Define Function to Calculate State of Health
def calculate_soh(discharge_capacity, nominal_capacity):
    return (discharge_capacity / nominal_capacity) * 100

#Calculate State of Health for Each Cell ID
soh = {cell_id: calculate_soh(capacity, nominal_capacity) for cell_id, capacity in discharge_capacities.items()}
print(soh)


{'5308': 99.734, '5329': 94.08533333333334}


In [5]:
pip install dash plotly

In [15]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


# import libraries to create the dashboard

In [7]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import requests
import plotly.express as px

C:\Users\ASUS\AppData\Local\Temp\ipykernel_36880\106017513.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\ASUS\AppData\Local\Temp\ipykernel_36880\106017513.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


# Initialize the app to create DashBoard

In [8]:
# Initialize the app
app = dash.Dash(__name__)


app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div([
        html.Div([
            html.H2('Dashboard', className='sidebar-header', style={'color': 'white'}),
            html.Button('Dashboard', id='dashboard-button', n_clicks=0, className='sidebar-button'),
            html.Button('Cell ID', id='cell-id-button', n_clicks=0, className='sidebar-button'),
            html.Div([
                dcc.Dropdown(
                    id='cell-dropdown',
                    options=[
                        {'label': 'Cell 5308', 'value': '5308'},
                        {'label': 'Cell 5329', 'value': '5329'}
                    ],
                    value='5308',
                    placeholder='Select Cell ID'  # Added placeholder
                ),
            ], id='dropdown-container', className='dropdown-container', style={'display': 'none'})
        ], className='sidebar', style={'background-color': 'blue'}),
        html.Div(id='page-content', className='content')
    ], className='main-container')
])

@app.callback(
    Output('dropdown-container', 'style'),
    [Input('cell-id-button', 'n_clicks')]
)
def toggle_dropdown_visibility(n_clicks):
    if n_clicks > 0:
        return {'display': 'block'}  # Show the dropdown
    return {'display': 'none'}  # Hide the dropdown

@app.callback(
    Output('page-content', 'children'),
    [Input('dashboard-button', 'n_clicks'), Input('cell-dropdown', 'value')]
)
def display_page(dashboard_n_clicks, cell_id):
    if dashboard_n_clicks > 0:
        return html.Div([
            html.H1('State of Health', style={'text-align': 'center'}),
            html.Div([
                dcc.Graph(
                    id='pie-chart-5308',
                    figure=px.pie(
                        names=['Healthy', 'Unhealthy'],
                        values=[soh['5308'], 100 - soh['5308']],
                        title='Cell 5308 SoH'
                    )
                ),
                dcc.Graph(
                    id='pie-chart-5329',
                    figure=px.pie(
                        names=['Healthy', 'Unhealthy'],
                        values=[soh['5329'], 100 - soh['5329']],
                        title='Cell 5329 SoH'
                    )
                )
            ], style={'display': 'flex', 'justify-content': 'space-around'})
        ])
    elif cell_id == '5308' or cell_id == '5329':
        return create_cell_visualizations(cell_id)
    return html.Div([html.H1('Welcome! Select a page from the sidebar.', style={'text-align': 'center'})])

def create_cell_visualizations(cell_id):
    # Select the data dictionary based on cell_id
    if cell_id == '5308':
        cell_data = dfs_5308
        detail_key = 'Detail'
        temp_key = 'DetailTemp'
    elif cell_id == '5329':
        cell_data = dfs_5329
        detail_key = 'Detail_67_3_1$'
        temp_key = 'DetailTemp_67_3_1$'
    else:
        return html.Div([html.H1("Invalid cell ID")])

    # Fetch the DataFrames based on the selected keys
    detail_df = cell_data.get(detail_key)
    temp_df = cell_data.get(temp_key)

    # Check if the DataFrames exist
    if detail_df is None:
        return html.Div([html.H1("Data not found for cell ID")])

    # Create visualizations
    fig1 = px.line(detail_df, x='Relative Time(h:min:s.ms)', y='Voltage(V)', title='Voltage vs Time')
    fig2 = px.line(detail_df, x='Relative Time(h:min:s.ms)', y='Cur(mA)', title='Current vs Time')
    fig3 = px.line(temp_df, x='Relative Time(h:min:s.ms)', y='Auxiliary channel TU1 T(°C)', title='Temperature vs Time')
    fig4 = px.line(detail_df, x='Relative Time(h:min:s.ms)', y='CapaCity(mAh)', title='Capacity vs Time')

    # Return the visualizations as a Div
    return html.Div([
        dcc.Graph(id=f'voltage-graph-{cell_id}', figure=fig1),
        dcc.Graph(id=f'current-graph-{cell_id}', figure=fig2),
        dcc.Graph(id=f'temperature-graph-{cell_id}', figure=fig3),
        dcc.Graph(id=f'capacity-graph-{cell_id}', figure=fig4)
    ])


# Start the Dash server in debug mode on port 8080 for local testing.
if __name__ == '__main__':
    app.run_server(debug=True, port=8080)

<center>
<h1 style="font-size: 50px; font-family: serif;">Thank You</h1>
</center>